In [ ]:
!git clone https://github.com/siddmuns/diabetes-ml-pipeline.git
%cd diabetes-ml-pipeline
!pip install -q -r requirements.txt

In [ ]:
!ls -l
# If you need to upload manually, use from google.colab import files; files.upload()

In [ ]:
import os
os.makedirs("artifacts", exist_ok=True)
os.makedirs("mlruns", exist_ok=True)

In [ ]:
from data_ingestion import preprocess_and_split
X_train, X_valid, X_test, y_train, y_valid, y_test, scaler, feature_names = preprocess_and_split(
    "diabetes.csv", out_dir="artifacts"
)

In [ ]:
import mlflow
mlflow.set_tracking_uri("file://" + "/content/mlruns")
mlflow.set_experiment("Diabetes_Pipeline")
print("MLflow tracking URI:", mlflow.get_tracking_uri())

In [ ]:
from tune_and_train import run_optuna, setup_mlflow, retrain_final_and_log
import numpy as np

# Ensure MLflow is set up
setup_mlflow(local_dir="mlruns", experiment_name="Diabetes_Pipeline")

# Load pre-saved splits
splits = np.load("artifacts/splits.npz", allow_pickle=True)
X_train = splits["X_train"]
X_valid = splits["X_valid"]
X_test = splits["X_test"]
y_train = splits["y_train"]
y_valid = splits["y_valid"]
y_test = splits["y_test"]

# Run Optuna hyperparameter search
study = run_optuna(X_train, y_train, X_valid, y_valid, n_trials=20)
best_params = study.best_trial.params
print("Optuna best params:", best_params)

# Retrain final model on train+valid
X_train_full = np.vstack([X_train, X_valid])
y_train_full = np.concatenate([y_train, y_valid])

import pandas as pd
example_input = pd.DataFrame(
    X_train_full[:5],
    columns=[f"feature_{i}" for i in range(X_train_full.shape[1])]
)

with mlflow.start_run(run_name="Final_Model") as run:
    model, test_acc, test_auc, train_time, model_path = retrain_final_and_log(
        X_train_full, y_train_full, X_test, y_test, best_params, artifacts_dir="artifacts"
    )
    mlflow.log_params(best_params)
    mlflow.log_metrics({
        "test_accuracy": float(test_acc),
        "test_roc_auc": float(test_auc),
        "train_time_s": float(train_time)
    })
    mlflow.sklearn.log_model(
        model,
        name="final_model",
        input_example=example_input
    )
    run_id = run.info.run_id
    print("Logged final model, run_id:", run_id)

In [ ]:
from mlflow import MlflowClient
client = MlflowClient()

# Skip registry if using file:// backend
if not mlflow.get_tracking_uri().startswith("file://"):
    try:
        model_uri = f"runs:/{run_id}/final_model"
        client.create_registered_model("Diabetes_GB_Model")
        mv = client.create_model_version("Diabetes_GB_Model", model_uri, run_id)
        print("Registered model version:", mv.version)
    except Exception as e:
        print("Model registration failed:", e)
else:
    print("Skipping registry on file:// backend")

In [ ]:
import joblib
joblib.dump(model, "artifacts/gb_final_model.pkl")
joblib.dump(scaler, "artifacts/scaler.pkl")
print("Saved gb_final_model.pkl and scaler.pkl")

In [ ]:
from batch_inference import batch_predict
out = batch_predict("artifacts/gb_final_model.pkl", "artifacts/scaler.pkl", "diabetes2.csv", "artifacts/predictions.csv")
out.head()

In [ ]:
from drift_detection import detect_and_log_drift
rpt = detect_and_log_drift("diabetes.csv", "diabetes2.csv", out_dir="artifacts")
print(rpt)

In [ ]:
from visualize_and_evaluate import make_plots_and_log
make_plots_and_log(model, X_test, y_test, feature_names, run_name="Final_Plots")